In [1]:
# =========================================================
# 📦 Cell 1 — Environment Setup and Global Configuration
# =========================================================

# Import basic libraries
from pathlib import Path
import sys, warnings
import numpy as np

# Set a global random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Configure Plotly renderer
# - "vscode" for VS Code Notebooks
# - "notebook_connected" for Jupyter
import plotly.io as pio
pio.renderers.default = "vscode"

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Print environment info for debugging
print("✅ Environment initialized")
print("Python version:", sys.version)
print("Plotly renderer:", pio.renderers.default)
print("Random seed:", RANDOM_SEED)

✅ Environment initialized
Python version: 3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang 16.0.0 (clang-1600.0.26.6)]
Plotly renderer: vscode
Random seed: 42


In [2]:
# =========================================================
# 🛠️ Cell 2 — Config Loader (YAML/JSON) with Safe Defaults
# =========================================================

from typing import Any, Dict
import json

# Try to import PyYAML. If not installed, we'll still work with JSON or defaults.
try:
    import yaml  # type: ignore
except Exception:
    yaml = None

def load_config(path: str | Path | None = None) -> Dict[str, Any]:
    """
    Load a configuration from YAML or JSON.
    Falls back to sensible defaults if the file is missing or YAML isn't installed.
    """
    # ---- Default configuration (edit these as needed) ----
    default = {
        "data": {
            # Primary location of your raw CSVs (you can change this to your local path)
            "input_dir": "/Users/dhwanil/Desktop/ONGOING PROJECTS/data for kpi",
            # Where to save the processed/cleaned dataset (used by the app later)
            "processed": "../data/processed/clean_data.csv",
        },
        "preprocess": {
            "grain": "global",         # "global" (aggregate all stores) or "store"
        },
        "models": {
            "ml": {
                "method": "isolation_forest",
                "contamination": 0.02,
                "n_estimators": 300,
                "random_state": RANDOM_SEED,
            }
        }
    }

    # If no path is provided, just return defaults
    if not path:
        return default

    p = Path(path)
    if not p.exists():
        print(f"⚠️ Config not found at {p}. Using defaults.")
        return default

    # Load YAML if available; else JSON
    if p.suffix.lower() in {".yaml", ".yml"} and yaml is not None:
        with open(p, "r", encoding="utf-8") as f:
            cfg = yaml.safe_load(f)
    elif p.suffix.lower() == ".json":
        with open(p, "r", encoding="utf-8") as f:
            cfg = json.load(f)
    else:
        print(f"⚠️ Unsupported config format for {p}. Using defaults.")
        return default

    # Shallow-merge file config over defaults
    def merge(a: dict, b: dict) -> dict:
        out = dict(a)
        for k, v in (b or {}).items():
            if isinstance(v, dict) and isinstance(out.get(k), dict):
                out[k] = merge(out[k], v)
            else:
                out[k] = v
        return out

    return merge(default, cfg or {})

# Load your config (change the path if needed)
CFG = load_config("configs/default.yaml")
print("✅ Config loaded")
print("Input dir:", CFG['data']['input_dir'])
print("Processed path:", CFG['data']['processed'])
print("Grain:", CFG['preprocess']['grain'])

⚠️ Config not found at configs/default.yaml. Using defaults.
✅ Config loaded
Input dir: /Users/dhwanil/Desktop/ONGOING PROJECTS/data for kpi
Processed path: ../data/processed/clean_data.csv
Grain: global
